### 심층 신경망 훈련하기

심층 신경망 훈련 도중 다음과 같은 상황을 마주할 수 있다.

* 까다로운 그레디언트 소실 또는 그레디언트 폭주 문제에 직면할 수 있음. 심층 신경망 아래쪽으로 갈수록 그레디언트가 점점 작아지거나 커지는 현상

* 대규모 신경망을 훈련하기 위한 데이터가 충분치 않거나 레이블을 만드는데 비용이 많이 듬

* 훈련이 극단적으로 느려짐

* 수백만개의 파라미터를 가진 모델은 과적합될 수 있음

### 그레디언트 소실, 폭주 문제

역전파 알고리즘이 하위층으로 진행될스록 그레디언트가 작아지거나 커지는 현상은 훈련을 어렵게 한다.

의심되는 원인으로서 많이 사용되는 로지스틱 시그모이드 활성화 함수와, 표준정규분포 가중치 초기화 방법이 거론된다.

로지스틱 함수는 입력인 양수나 음수로 커지면 기울기가 0에 매우 가까워진다.

역전파가 될 때, 전달될 그레디언트가 거의 없고, 조금 있는 그레디언트는 최상위층으로부터 진행되며 점점 약해진다.

#### 가중치 초기화 방법

로지스틱 함수를 이용할 때, 각 층의 연결 가중치를 아래의 식대로 무작위로 초기화하면 그레디언트 문제를 완화할 수 있다.

이를 Xavier initialization라고 한다.

$$ \text{Normal distribution with mean 0 and variance } \sigma = \sqrt{\frac{1}{fan_{\mathrm{avg}}}} $$
$$ \text{ or a uniform distribution between } -r \text{ and } r, \text{ with } r = \sqrt{\frac{2}{fan_{\mathrm{avg}}}} $$

여기서 $fan_{\mathrm{avg}}$은 층의 입력과 출력의 연결 개수인 $fan_{\mathrm{in}}$과 $fan_{\mathrm{out}}$의 평균이다.

그 외에 다음과 같은 초기화 전략들이 있다.

  
| 초기화 전략 | 활성화 함수 | 정규분포 분산 |
|--|--|--|
| Glorot | 활성화 함수 없음, 하이퍼볼릭 탄젠트, 로지스틱, 소프트맥스 | $1/fan_{\mathrm{avg}}$ |
| He | ReLu 함수와 그 변종들 | $2 / fan_{\mathrm{in}}$ |
| LuCun | SELU | $1/fan_{\mathrm{in}}$ |

케라스는 기본적으로 균등분포의 글로럿 초기화를 사용한다. 

다음과 같이 층을 만들 때, ```kernerl_initializer="he_uniform"```로 바꾸어 쓸 수 있다. 

``` keras.layer.Dense(10, activation="relu", kernel_initializer="he_normal")```


#### 활성화 함수 문제

활성화 함수를 잘못 선택하면 그레디언트 소실이나 폭주로 이어질 수 있다.

초기에는 생물학적 뉴런과 비슷한 시그모이드 함수가 잘 작동할 것이라고 생각되었지만, ReLu 같은 활성화 함수가 심층 신경망에서 더 잘 작동하는 경향이 있다.

하지만 ReLu도 완벽하지 않고, 죽은 ReLu 문제가 알려져 있다.

훈련 중 일부 뉴런이 0만 출력하는 현상이다.

이 문제를 해결하기 위해 LeakyReLu, ELU, SELU 같은 변종을 사용한다.

$$ \mathrm{LeakyReLu}_{\alpha}(z) = \max(\alpha z, z) $$

$$ \mathrm{ELU}_{\alpha}(z) = 
\left\{ \begin{array}{ll} \alpha (\exp(z) - 1) & z < 0 \\
z & z \geq 0 \end{array} \right.
$$

케라스에서 LeakyReLu를 사용하려면 다음과 같이 코드를 작성한다.

```
model = keras.models.Sequential([
  [...]
  keras.layers.Dense(10, kernel_initializer="he_normal"),
  keras.layers.LeakyReLU(alpha=0.2)
  [...]
])
```

SELU 함수를 사용하려면 다음과 같이 코드를 작성한다.

``` lyaer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal") ```

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [6]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [7]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.6314 - accuracy: 0.5054 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.8416 - accuracy: 0.7247 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.7053 - accuracy: 0.7638 - val_loss: 0.6427 - val_accuracy: 0.7900
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6325 - accuracy: 0.7908 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5992 - accuracy: 0.8020 - val_loss: 0.5582 - val_accuracy: 0.8200
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5624 - accuracy: 0.8141 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5379 - accuracy: 0.8217 - val_loss: 0.5156 - val_accuracy:

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.6965 - accuracy: 0.4881 - val_loss: 0.8951 - val_accuracy: 0.7194
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.8559 - accuracy: 0.7299 - val_loss: 0.7103 - val_accuracy: 0.7742
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.7083 - accuracy: 0.7699 - val_loss: 0.6404 - val_accuracy: 0.7958
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6353 - accuracy: 0.7901 - val_loss: 0.5859 - val_accuracy: 0.8128
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6011 - accuracy: 0.8011 - val_loss: 0.5542 - val_accuracy: 0.8192
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5645 - accuracy: 0.8133 - val_loss: 0.5314 - val_accuracy: 0.8242
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5398 - accuracy: 0.8181 - val_loss: 0.5121 - val_accuracy:

#### 배치 정규화

각 층에서 활성화 함수를 통과하기 전이나 후에 정규화 연산을 추가한다.

이 연산은 단순히 입력을 원점에 맞추어 정규화한 다음, 각 층에서 두 개의 새로운 파라미터로 결과값을 스케일링하고 이동시킨다.

미니배치별 배치정규화층에 사용될 입력값의 표본평균과 표본표준편차를 계산한다.

평균과 표준편차를 이용하여 입력값을 정규화한다. 

배치정규화층에는 스케일 파라미터 $\mathbf{\gamma}$와 오프셋 파라미터 $\mathbf{\beta}$가 있어 이를 이용해 배치 정규화 출력값을 계산한다.

$$ \mathbf{\gamma} \circ \mathbf{\hat x} + \mathbf{\beta} $$

배치정규화는 신경망의 성능을 크게 향상시킨다고 알려져 있다.

케라스에서는 다음의 코드로 배치 정규화를 구현한다.

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_8 (Dense)              (None, 10)               

In [13]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [14]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [15]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.1669 - accuracy: 0.6144 - val_loss: 0.5548 - val_accuracy: 0.8134
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6000 - accuracy: 0.7937 - val_loss: 0.4757 - val_accuracy: 0.8364
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5274 - accuracy: 0.8153 - val_loss: 0.4405 - val_accuracy: 0.8524
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4883 - accuracy: 0.8306 - val_loss: 0.4185 - val_accuracy: 0.8580
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4721 - accuracy: 0.8356 - val_loss: 0.4025 - val_accuracy: 0.8616
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4426 - accuracy: 0.8432 - val_loss: 0.3908 - val_accuracy: 0.8670
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4282 - accuracy: 0.8508 - val_loss: 0.3807 - val_accuracy:

In [16]:
#### 활성화 함수 전에 사용하는 방법


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [18]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.3856 - accuracy: 0.5573 - val_loss: 0.6611 - val_accuracy: 0.7864
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6956 - accuracy: 0.7784 - val_loss: 0.5478 - val_accuracy: 0.8196
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6015 - accuracy: 0.8004 - val_loss: 0.4968 - val_accuracy: 0.8332
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5469 - accuracy: 0.8162 - val_loss: 0.4662 - val_accuracy: 0.8412
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5213 - accuracy: 0.8204 - val_loss: 0.4438 - val_accuracy: 0.8498
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4883 - accuracy: 0.8311 - val_loss: 0.4278 - val_accuracy: 0.8528
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4733 - accuracy: 0.8363 - val_loss: 0.4141 - val_accuracy:

#### 그레이디언트 클리핑

그레디언트 폭주 문제를 완화하는 방법으로 역전파 수행 시 그레디언트가 일정 임계값을 넘지 못하도록 하는 것이다.

```optimizer = keras.optimizers.SGD(clipvalue=1.0)```

```optimizer = keras.optimizers.SGD(clipnorm=1.0)```

### 훈련된 층 재사용 하기

아주 큰 DNN을 처음부터 훈련하는 것인 시간이 오래 걸리기 때문에 비슷한 유형의 문제를 처리한 신경망의 하위층을 재사용할 수 있다.

(상위 층은 덜 유용함)

이를 전이 학습 (transfer learning)이라 한다.

먼저 재사용 층의 가중치들을 동결한다.

그 다음 모델을 학습하고 평가한다.

맨 위 한 두개의 은닉층의 동결을 해제하고 역전파를 통해 가중치를 조정하여 성능이 향상되는지 확인한다.

#### 케라스를 이용한 전이 학습

패션 MNIST 훈련 세트를 두 개로 나눈다.

* X_train_A: 샌달과 셔츠(클래스 5와 6)을 제외한 모든 이미지
* X_train_B: 샌달과 셔츠 이미지 중 처음 200개만 가진 작은 훈련 세트


A 세트(8개의 클래스를 가진 분류 문제)에서 모델을 훈련하고 이를 재사용하여 B 세트(이진 분류)를 해결해 보자. 

A 세트의 클래스(스니커즈, 앵클 부츠, 코트, 티셔츠 등)가 B 세트에 있는 클래스(샌달과 셔츠)와 조금 비슷함. 

In [19]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [20]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [21]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [22]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 4s 2ms/step - loss: 0.8501 - accuracy: 0.7172 - val_loss: 0.3924 - val_accuracy: 0.8592
Epoch 2/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3753 - accuracy: 0.8722 - val_loss: 0.3317 - val_accuracy: 0.8884
Epoch 3/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3251 - accuracy: 0.8888 - val_loss: 0.3038 - val_accuracy: 0.8959
Epoch 4/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3109 - accuracy: 0.8929 - val_loss: 0.2891 - val_accuracy: 0.9008
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2889 - accuracy: 0.8980 - val_loss: 0.2793 - val_accuracy: 0.9028
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2771 - accuracy: 0.9048 - val_loss: 0.2728 - val_accuracy: 0.9053
Epoch 7/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2699 - accuracy: 0.9075 - val_loss: 0.2731 - val_accuracy:

In [23]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [24]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [25]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 44ms/step - loss: 0.9967 - accuracy: 0.5075 - val_loss: 0.5225 - val_accuracy: 0.7515
Epoch 2/20
7/7 [==============================] - 0s 17ms/step - loss: 0.4764 - accuracy: 0.7900 - val_loss: 0.4004 - val_accuracy: 0.8408
Epoch 3/20
7/7 [==============================] - 0s 15ms/step - loss: 0.3496 - accuracy: 0.8775 - val_loss: 0.3317 - val_accuracy: 0.8763
Epoch 4/20
7/7 [==============================] - 0s 16ms/step - loss: 0.2798 - accuracy: 0.9325 - val_loss: 0.2840 - val_accuracy: 0.9047
Epoch 5/20
7/7 [==============================] - 0s 17ms/step - loss: 0.2451 - accuracy: 0.9275 - val_loss: 0.2522 - val_accuracy: 0.9239
Epoch 6/20
7/7 [==============================] - 0s 16ms/step - loss: 0.2217 - accuracy: 0.9417 - val_loss: 0.2289 - val_accuracy: 0.9351
Epoch 7/20
7/7 [==============================] - 0s 16ms/step - loss: 0.1759 - accuracy: 0.9666 - val_loss: 0.2121 - val_accuracy: 0.9351
Epoch 8/20
7/7 [===========

In [26]:
model_B.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_19 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_20 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_21 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_22 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                